In [1]:
from haystack.schema import Document
from haystack.document_stores import InMemoryDocumentStore
import rocketqa
from rocket_qa_retriever import RocketQAEmbeddingRetriever
import threading

model = rocketqa.load_model(model="zh_dureader_de", use_cuda=False)

def get_doc_retriever(document_store=None):
    retriever = RocketQAEmbeddingRetriever(
        model=model,
        document_store=document_store,
        use_gpu=False,
        is_faq=False,
        scale_score=False,
    )
    return retriever

doc_retriever = get_doc_retriever() # 文档检索器，全局变量

locker = threading.Lock()

document_store = InMemoryDocumentStore() 

c:\Users\zhushatong\.conda\envs\chatlongdoc\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
RocketQA model [zh_dureader_de]
Download RocketQA model [zh_dureader_de]
100%|███████████████████████████████████████| 692M/692M [09:34<00:00, 1.26MiB/s]
Load model done


In [2]:
document_store = InMemoryDocumentStore() 

text = """
春季是万物复苏的季节，花开得特别美。在这个季节里，人们喜欢去公园赏花、踏青。
人工智能是当前技术革命的热门领域，它涉及机器学习、深度学习等多个技术方向。随着研究的深入，人工智能正逐步走入人们的生活。
古代中国的四大发明——造纸术、火药、印刷术、指南针，对世界文明的发展产生了深远的影响。
健康饮食对于维持良好的身体状态和预防疾病至关重要。均衡的饮食应包括蔬菜、水果、全谷物等多种食物。
星空下的露营是一种无与伦比的体验，远离城市的喧嚣，人们可以更接近自然，感受宁静和浩瀚。
区块链技术正在逐渐改变我们的金融生态系统，它提供了一种安全、去中心化的交易方式。
茶文化在中国有着悠久的历史，不仅是饮品的享受，也是精神文化的一部分。
旅行不仅能拓宽视野，更能让人心灵得到放松和充实，每次的旅行都是一次新的探索和发现。
电子竞技已成为全球范围内的热门赛事，吸引了无数年轻人的关注和参与。
环境保护已经成为全球性的紧迫议题，减少碳排放和保护生态多样性是我们共同的责任。
传统节日不仅是对历史和文化的纪念，也是现代社会中强化社区关系和家庭纽带的重要方式。
人工智能与机器学习的结合正在开启医疗健康领域的新篇章，它们的应用有望带来更准确的诊断和个性化的治疗方案。
城市化进程带来了便利的同时，也引发了诸如交通拥堵、空气污染等问题，寻求可持续发展的解决方案变得尤为关键。
跨文化交流促进了全球化进程，理解和尊重不同文化的价值观是实现和谐共处的基础。
"""

segment_list = text.replace("\n", " ").split()

for i, content in enumerate(segment_list):
    # 生成嵌入向量
    locker.acquire()
    embedding = doc_retriever.embed_documents([Document(content=content)])[0] # 使用retriever生成嵌入向量
    locker.release()

    embedding = embedding.tolist()
    print(embedding)

    doc1 = Document(id=i, content=content, embedding=embedding)

    document_store.write_documents([doc1])

[0.009350703097879887, -0.01935325376689434, -0.010587317869067192, 0.0018499185098335147, 0.0027180579490959644, 0.010910059325397015, 0.0011093458160758018, 0.008160734549164772, -0.006521392613649368, 0.02253538742661476, 0.011411390267312527, -0.0027206665836274624, -0.008036886341869831, -0.024221345782279968, 0.006416039541363716, 0.013761424459517002, -0.015665972605347633, 0.0020674392580986023, -0.007148200646042824, -0.017788570374250412, 0.024301432073116302, 0.04888778552412987, -0.021558852866292, 0.02587498165667057, -0.0030037539545446634, 0.012662243098020554, 0.004202317912131548, 0.03610929474234581, 0.02159414254128933, 0.004593017976731062, 0.007504389155656099, -0.01247798465192318, 0.003995384089648724, 0.0007635362562723458, 0.019955001771450043, 0.029039183631539345, 0.006799053866416216, -0.014051517471671104, -0.006344271823763847, 0.55491042137146, -0.018342051655054092, -0.019316814839839935, -0.03327080234885216, -0.0030581930186599493, -0.04548763483762741

In [3]:
MAX_TOP_N = 2

query = "了解一下文化"

locker.acquire()
documents = doc_retriever.retrieve(query,
                                    top_k=MAX_TOP_N,
                                    document_store=document_store)
print(f'Got search_doc_embedding_data count: {len(documents)}')
locker.release()

Got search_doc_embedding_data count: 2


In [4]:
# 打印检索结果
for doc in documents:
    print(f"doc_id: {doc.id}, content: {doc.content}, score: {doc.score}")

doc_id: 13, content: 跨文化交流促进了全球化进程，理解和尊重不同文化的价值观是实现和谐共处的基础。, score: 0.7684352888377398
doc_id: 6, content: 茶文化在中国有着悠久的历史，不仅是饮品的享受，也是精神文化的一部分。, score: 0.7679683413412494
